In [43]:
import os,sys; sys.path.insert(0,os.path.abspath('../../..'))
from epistolary import *

In [41]:
t = Text('hello', corpus='chadwyck')

[16:48:34] lltk.text.text.Text() > Text(id="hello", corpus="chadwyck"
[16:48:34] lltk.corpus.corpus.Corpus() > Corpus(chadwyck, **{})
[16:48:34] lltk.corpus.corpus.Corpus() > Corpus() called on a string: "chadwyck"
[16:48:34] lltk.corpus.corpus.Corpus() > "chadwyck" found, returning: <Chadwyck> chadwyck (Chadwyck)
[16:48:34] lltk.text.text.Text() > got corpus: <Chadwyck> chadwyck (Chadwyck)
[16:48:34] lltk.corpus.corpus.init_text() > id=hello, corpus=None, meta={'id': 'hello'}, **{})
[16:48:34] lltk.text.text.Text() > returning text <TextChadwyck> Author, “Title” (Year) [Chadwyck: hello]


In [42]:
C=Corpus('chadwyck')

[16:48:35] lltk.corpus.corpus.Corpus() > Corpus(chadwyck, **{})
[16:48:35] lltk.corpus.corpus.Corpus() > Corpus() called on a string: "chadwyck"
[16:48:35] lltk.corpus.corpus.Corpus() > "chadwyck" found, returning: <Chadwyck> chadwyck (Chadwyck)


>> [16:17:07] lltk.corpus.corpus.Corpus():
	Corpus() called on a string: "chadwyck"

>> [16:17:07] lltk.corpus.corpus.Corpus():
	"chadwyck" found, returning: <Chadwyck> chadwyck (Chadwyck)



#### t = C.text()

>> [16:17:07] lltk.corpus.corpus.init_text():
	id=None, corpus=None, meta={'id': 'hello'}, **{})

>> [16:17:07] lltk.text.text.Text():
	called on a text object: <TextChadwyck> Author, _Title_ (Year) [Chadwyck: ID]

>> [16:17:07] lltk.text.text.Text():
	no corpus_ref set, returning as-is: <TextChadwyck> Author, _Title_ (Year) [Chadwyck: ID]




* ID: None
* name: None
* corpus: <Chadwyck> chadwyck (Chadwyck)

* corpus copy: <TextChadwyck> Author, _Title_ (Year) [Chadwyck: ID]


#### t = Text("id")

...

>> [16:17:07] lltk.text.text.Text():
	called on a text object: <BaseText> Author, _Title_ (Year) [TmpCorpus: plain_text]

>> [16:17:07] lltk.text.text.Text():
	no corpus_ref set, returning as-is: <BaseText> Author, _Title_ (Year) [TmpCorpus: plain_text]

>> [16:17:07] lltk.corpus.corpus.init_text():
	id=None, corpus=None, meta={'id': 'plain_text'}, **{})




* ID: plain_text
* name: None
* corpus: <BaseCorpus> tmp_corpus (TmpCorpus)

* corpus copy: None


#### t = Text("_corpus/id")

>> [16:17:07] lltk.text.text.Text():
	getting text with text_ref=with_corpus_ref and corpus_ref=chadwyck

>> [16:17:07] lltk.corpus.corpus.Corpus():
	Corpus() called on a string: "chadwyck"

>> [16:17:07] lltk.corpus.corpus.Corpus():
	"chadwyck" found, returning: <Chadwyck> chadwyck (Chadwyck)

>> [16:17:07] lltk.text.text.Text():
	got corpus: <Chadwyck> chadwyck (Chadwyck)

>> [16:17:07] lltk.corpus.corpus.init_text():
	id=with_corpus_ref, corpus=None, meta={'id': None}, **{})

>> [16:17:07] lltk.text.text.Text():
	got text: <TextChadwyck> Author, _Title_ (Year) [Chadwyck: with_corpus_ref]




* ID: with_corpus_ref
* name: None
* corpus: <Chadwyck> chadwyck (Chadwyck)

* corpus copy: None


In [14]:
t2 = Text('_chadwyck/hello2')



Author?, _Title?_ (Year?) [Chadwyck: hello2] <chadwyck.Chadwyck object at 0x107e30c10>


KeyError: 'hello2'

In [6]:
# resetindex(C.meta)

In [7]:
# def init(self, meta_final=None, other_meta=[], expand_meta=True, force=False,**kwargs):
#     # metas
#     meta_objs=[
#         self.path_metadata_int,
#         self.path_metadata,
#         self._metadf if not force else None,
#     ]

#     # first build objects
#     for t in 

    
#     # so far?
#     meta_l=[]
#     # local too?
#     if self.path_metadata_init and os.path.exists(self.path_metadata_init):
#         meta_l.append(self.path_metadata_init)
    
#     # fn?
#     if self.path_metadata and os.path.exists(self.path_metadata):
#         meta_l.append(self.path_metadata)
    
#     # already?
#     if not force and self._metadf is not None:
#         meta_l.append(self._metadf)
    
#     # others?
#     meta_l.extend(other_meta)

#     # final?
#     if meta_final is not None: meta_l.append(meta_final)

#     meta=None
#     for mdf_or_fn in meta_l:
#         # display('MERGING',mdf_or_fn)

#         mdf = load_metadata_from_df_or_fn(mdf_or_fn,**kwargs)
#         # display(mdf)

#         if meta is None:
#             meta=mdf
#         else:
#             for col in set(mdf.columns) - set(meta.columns): meta[col]=''
#             meta.update(mdf)

#         # display(meta,'\n\n\n\n\n\n')

    
#     self._metadf = meta

#     if meta is not None: 
#         meta = meta.fillna('')
#         text_il=meta.index
#         text_ld=meta.reset_index().to_dict(orient='records')
#         self._texts=[
#             self.init_text(id=idx,meta=text_d)
#             for idx,text_d in zip(
#                 meta.index,
#                 meta.to_dict(orient='records')
#             )
#         ]
#         self._textd=dict(zip(meta.index, self._texts))

#         if expand_meta:
#             meta = self._metadf = load_metadata_from_df_or_fn(
#                 pd.DataFrame([t.meta for t in self._texts])
#             )
    
#     return self._metadf
    

SyntaxError: invalid syntax (4208883767.py, line 10)

In [9]:
init(C)

,author,title,year,idref,pub,type,name,nation,medium,subcorpus,fn_raw
id,,,,,,,,,,,
Eighteenth-Century_Fiction/paltock.01,"Paltock, Robert, 1697-1767",Peter Wilkins (1751),1751,Z200043982,"Printed for J. Robinson, ... and R. Dodsley, [...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/paltock.01.new
Eighteenth-Century_Fiction/brookefm.02,"Brooke, Frances, 1724?-1789",Lady Julia Mandeville (1763),1763,Z200000703,Printed for R. and J. Dodsley [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/brookefm.02.new
Eighteenth-Century_Fiction/mackenzi.01,"Mackenzie, Henry, 1745-1831",Julia de Roubign&eacute; (1777),1777,Z200030400,"Printed for W. Strahan, and T. Cadell",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/mackenzi.01.new
Eighteenth-Century_Fiction/cleland.01,"Cleland, John, 1709-1789",Memoirs of a Woman of Pleasure (1749),1749,Z200000927,Printed for G. Fenton [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/cleland.01.new
Eighteenth-Century_Fiction/haywood.07,"Haywood, Eliza Fowler, 1693?-1756",The Fortunate Foundlings (1744),1744,Z200029933,Printed and published by T. Gardner [etc.],BOOK,Title Page,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/haywood.07.new
...,...,...,...,...,...,...,...,...,...,...,...
Early_English_Prose_Fiction/ee54010.01,"Holland, Samuel, gent.",Don Zara Del Fogo (1656),1656,Z200027421,Printed by T. W. for Tho. Vere [etc.],BOOK,Title Page,British,Fiction,Early_English_Prose_Fiction,Early_English_Prose_Fiction/ee54010.01.new
Early_English_Prose_Fiction/ee17010.04,"Orrery, Roger Boyle, Earl of, 1621-1679","Parthenissa, Part 3 (1655)",1655,Z200025922,Printed for Humphrey Moseley [etc.],BOOK,Title Page,British,Fiction,Early_English_Prose_Fiction,Early_English_Prose_Fiction/ee17010.04.new
Early_English_Prose_Fiction/ee73020.01,"Oldys, Alexander",The Female Gallant (1692),1692,Z200027810,Printed for Samuel Briscoe [etc.],BOOK,Title Page,British,Fiction,Early_English_Prose_Fiction,Early_English_Prose_Fiction/ee73020.01.new


In [6]:
gen=iter_metadata_from_df_or_fn(C.meta)
next(gen)

{'id': 'Eighteenth-Century_Fiction/paltock.01',
 'author': 'Paltock, Robert, 1697-1767',
 'title': 'Peter Wilkins (1751)',
 'year': '1751',
 'idref': 'Z200043982',
 'pub': 'Printed for J. Robinson, ... and R. Dodsley, [etc.]',
 'type': 'BOOK',
 'name': 'Vol. 1',
 'nation': 'British',
 'medium': 'Fiction',
 'subcorpus': 'Eighteenth-Century_Fiction',
 'fn_raw': 'Eighteenth-Century_Fiction/paltock.01.new'}

In [15]:
c.textd['hello']

KeyError: 'hello'

In [14]:
c.init()

c.textd[CLAR_IDX]

Richardson, Samuel, 1689-1761, _Clarissa (1st ed.) (1748)_ (1748) [Chadwyck: Eighteenth-Century_Fiction/richards.01]

In [10]:
meta_iter_corpora??

Signature: meta_iter_corpora(corpora, **y)
Docstring: <no docstring>
Source:   
def meta_iter_corpora(corpora,**y):
    for C in to_corpus_objs(corpora):
        yield from C.meta_iter(**y)
File:      ~/github/lltk/lltk/corpus/utils.py
Type:      function


In [15]:
c.text??

Signature: c.text(id=None, **kwargs)
Docstring: <no docstring>
Source:   
    def text(self,id=None,**kwargs):
        if id is not None and id in self.textd:
            return self._textd[id]
        else:            
            obj=self.init_text(id=id,**kwargs)
            if obj is not None: self._textd[id]=obj
            return obj
File:      ~/github/lltk/lltk/corpus/corpus.py
Type:      method


Signature: c.init_text(id=None, corpus=None, meta={'id': 'testing'}, **kwargs)
Docstring: <no docstring>
Source:   
    def init_text(self,id=None,corpus=None,meta={},**kwargs):
        corpus = self if corpus is None else corpus
        t=self.TEXT_CLASS(
            id=id,
            corpus=corpus,
            meta=meta,
            **kwargs
        )
        # if type(self._textd)!={}: self._textd={}
        # self._textd[id]=t
        return t
File:      ~/github/lltk/lltk/corpus/corpus.py
Type:      method


In [3]:
t.corpus.name, t.id

('TmpCorpus', 'testing')

In [4]:
# t=Text()
# t=Text('hello')
t=Text(CLAR_ID)

print(t)
print(t.id, type(t))
print(t.corpus.id, t.corpus.name, type(t))

Author?, _Title?_ (Year?) [Chadwyck: Eighteenth-Century_Fiction/richards.01]
Eighteenth-Century_Fiction/richards.01 <class 'chadwyck.TextChadwyck'>
chadwyck Chadwyck <class 'chadwyck.TextChadwyck'>


In [5]:
t=Text('hello')

In [4]:
type(c.text('hello'))

chadwyck.TextChadwyck

In [11]:
clar = Text(CLAR_ID)
clar.corpus

In [14]:
type(clar)

chadwyck.TextChadwyck

In [16]:
Text('testing testing')

TypeError: 'NoneType' object does not support item assignment

In [37]:
x = Text('testing testing')

In [38]:
x.corpus

In [19]:
type(t.corpus)

chadwyck.Chadwyck

In [4]:
c=Corpus('chadwyck')
c.id, c.name, type(c)

('chadwyck', 'Chadwyck', chadwyck.Chadwyck)

In [6]:
Corpus(c)

In [8]:
Corpus(c) is c

True

In [ ]:
c.XML

In [21]:
c = Corpus('cref')

In [24]:
c.idacef

AttributeError: 'Corpus' object has no attribute 'idacefe'

In [17]:
t.path_texts

In [11]:
c.path_root

'/Users/ryan/lltk_data/corpora/tmp_corpus'

In [3]:
clarid = get_clarissa_id()
t =clar = get_clarissa()
clar

Richardson, Samuel, 1689-1761, _Clarissa (1st ed.) (1748)_ (1748) [Chadwyck: Eighteenth-Century_Fiction/richards.01]

In [4]:
t=Text(id=clarid)

In [6]:
t.corpus

In [10]:
class TextEpistolary(Text):

    def __init__(self):
        self.path_te

    def compile(self,lim=None,progress=True,force=False,*x,**y):
        ofn_ltrs = os.path.join(C.paths['path_letters'], t.corpus.id, t.id + '.csv')
        ofn_xml = os.path.join(C.paths['path_xml'], t.corpus.id, t.id + '.xml')
        ofn_txt = os.path.join(C.paths['path_txt'], t.corpus.id, t.id + '.txt')

        if not force and os.path.exists(ofn_ltrs):
            odf = read_df(ofn_ltrs)
            odf_anno=load_with_anno(ofn_ltrs)
            if len(odf_anno) and 'id' in set(odf_anno.columns):
                odf_anno=odf_anno.set_index('id')
                odf_anno=odf_anno[[col for col in odf_anno.columns if not col.startswith('id_')]]
                odx_anno=dict((idx, dict(row)) for idx,row in odf_anno.iterrows())

                newkeys=set(rowdk for idx,rowd in odx_anno.items() for rowdk in rowd)
                for nk in newkeys:
                    odf[nk]=[odx_anno.get(idx,{}).get(nk,'') for idx in odf.id]
            
            return fix_meta(odf)
            

        ol=[]
        #if force or not os.path.exists(ofn_ltrs) or not os.path.exists(ofn_xml) or not os.path.exists(ofn_txt):
        if not os.path.exists(os.path.dirname(ofn_xml)): os.makedirs(os.path.dirname(ofn_xml))
        if not os.path.exists(os.path.dirname(ofn_txt)): os.makedirs(os.path.dirname(ofn_txt))            
        if not os.path.exists(os.path.dirname(ofn_ltrs)): os.makedirs(os.path.dirname(ofn_ltrs))            
        
        tdf=epistolarized_chadwyck_t(C,t=t,lim=lim,progress=progress,*x,**y)
        if not len(tdf): return pd.DataFrame()

    
        todf=fix_meta(tdf)

        for pxml,txml in zip(tdf.path_xml,tdf.xml):
            if pxml and txml:
                with open(pxml,'w') as of: of.write(txml)
        for ptxt,ttxt in zip(tdf.path_txt,tdf.txt):
            if ptxt and ttxt:
                with open(ptxt,'w') as of: of.write(ttxt)

        for needcol in {'sender_tok','sender_id','recip_tok','recip_id','front','title_letter'}:
            if not needcol in set(todf.columns): todf[needcol]=''
        for badcol in {'txt','xml','path_txt','path_xml'} & set(todf.columns):
            todf=todf.drop(badcol,1)
        
        cols=[
            'id','id_corpus','id_text','id_letter','letter_i',
            'txt_head',
            'txt_front',
            'sender_tok',#'sender_id',
            'recip_tok',#'recip_id',
        ]
        odf_ltrs=todf[[c for c in cols if c in set(todf.columns)]].fillna('')
        save_df(fix_meta(odf_ltrs), ofn_ltrs, verbose=True)
        return odf_ltrs

        
    def compile_text_chars(self,t,tdf=None,force=False,force_inner=False,verbose=False,*x,**y):
        id_corpus=t.corpus.id
        id_text=t.id

        ofn_tok2id = os.path.join(self.path_chars, id_corpus, id_text, 'tok2id.csv')
        ofn_id2meta = os.path.join(self.path_chars, id_corpus, id_text, 'id2meta.csv')

        if force or not os.path.exists(ofn_tok2id):
            # get latest sender,recip data
            if tdf is None: tdf=self.compile_text(t,force=force_inner,*x,**y)
            odf_tok2id=calculate_tok2id(tdf.fillna(''))
            save_df(odf_tok2id, ofn_tok2id, verbose=verbose, index=False)
            if verbose: display(odf_tok2id)
        else:
            odf_tok2id=read_df(ofn_tok2id)
        
        # update with anno
        odf_tok2id_anno=load_with_anno(ofn_tok2id)
        if len(odf_tok2id_anno):
            d_tok2id_anno=dict(zip(odf_tok2id_anno.char_tok, odf_tok2id_anno.char_id)) if len(odf_tok2id_anno) else {}
            odf_tok2id['char_id']=[d_tok2id_anno.get(ctok,cid) for ctok,cid in zip(odf_tok2id.char_tok,odf_tok2id.char_id)]
        
        

        ## id2meta
        if force or not os.path.exists(ofn_id2meta):
            ## id2meta
            char_ids = Counter()
            for char_id,char_tok_count in zip(odf_tok2id.char_id, odf_tok2id.char_tok_count):
                char_ids[char_id]+=char_tok_count
            # init?
            id2meta_l=[]
            for char_id,char_id_count in char_ids.most_common():
                char_dx={'char_id':char_id, 'char_id_count':char_id_count, **chardata_metakeys_initial}
                id2meta_l.append(char_dx)
            odf_id2meta = pd.DataFrame(id2meta_l).fillna('')
            save_df(odf_id2meta, ofn_id2meta, verbose=verbose, index=False)
            if verbose: display(odf_id2meta)
        else:
            odf_id2meta=read_df(ofn_id2meta)

        # update with anno
        odf_id2meta=odf_id2meta.set_index('char_id')
        odf_id2meta_anno=load_with_anno(ofn_id2meta)
        if len(odf_id2meta_anno) and 'char_id' in set(odf_id2meta_anno.columns):
            odf_id2meta_anno=odf_id2meta_anno.fillna('').set_index('char_id')
            odf_id2meta_anno=odf_id2meta_anno[[col for col in odf_id2meta_anno if col not in {'char_id_count'}]]
            odf_id2meta.update(odf_id2meta_anno)
        odf_id2meta=odf_id2meta.reset_index()
        
        return odf_tok2id.fillna(''), odf_id2meta.fillna('')
        
    def get_letters(self,t,force=False,*x,**y):
        df_letters=self.compile_text(t,force=force,*x,**y)
        df_tok2id,df_id2meta=self.compile_text_chars(t,force=force,*x,**y)
        d_tok2id=dict(zip(df_tok2id.char_tok, df_tok2id.char_id))
        cols_id2meta_uniq=[col for col in df_id2meta if col not in set(df_letters.columns)]
        df_id2meta=df_id2meta[cols_id2meta_uniq]

        for sndr in ['sender','recip']:
            sidkey=f'{sndr}_id'
            df_letters[sidkey]=df_letters[f'{sndr}_tok'].apply(lambda stok: d_tok2id.get(stok,stok))
            senders = set(df_letters[sidkey])
            df_id2meta_now = df_id2meta[df_id2meta.char_id.isin(senders)]
            df_id2meta_now.columns=[col.replace('char_',f'{sndr}_') for col in df_id2meta.columns]
            if sidkey in set(df_id2meta_now.columns):
                df_letters = df_letters.merge(df_id2meta_now, how='left', on=sidkey)
            
        return df_letters


    def iter_letter_networks(self,t,dfletters=None,*x,**y):
        dfletters=self.get_letters(t,*x,**y) if dfletters is None else dfletters
        return iter_letter_networks_from_dfletters(dfletters,*x,**y)

    def get_letter_network(self,t,dfletters=None,*x,**y):
        dfletters=self.get_letters(t,*x,**y) if dfletters is None else dfletters
        return get_letter_network_from_dfletters(dfletters,*x,**y)







class Epistolary(Corpus):
    NAME='Epistolary'
    ID='epistolary'
    TEXT_CLASS=TextEpistolary
    CORPORA_TO_START_WITH = ['Chadwyck']


# C = Epistolary()
# C.meta

In [5]:
C=Epistolary()
C.init()

,testing,author,title,year,idref,pub,type,name,nation,medium,subcorpus,fn_raw
id,,,,,,,,,,,,
Eighteenth-Century_Fiction/richards.02,x,,,,,,,,,,,
_chadwyck/Eighteenth-Century_Fiction/richards.04,y,"Richardson, Samuel, 1689-1761",Pamela (1741),1741,Z200045357,"Printed for C. Rivington, in St. Paul's Church...",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/richards.04.new
_chadwyck/Eighteenth-Century_Fiction/burney.01,,"Burney, Fanny, 1752-1840",Evelina (1778),1778,Z200000830,"Printed for T. Lowndes, [etc.]",BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/burney.01.new


In [6]:
t=C.texts()[-1]
t, t.source

(Burney, Fanny, 1752-1840, _Evelina (1778)_ (1778) [Epistolary: _chadwyck/Eighteenth-Century_Fiction/burney.01],
 Burney, Fanny, 1752-1840, _Evelina (1778)_ (1778) [Chadwyck: Eighteenth-Century_Fiction/burney.01])

In [8]:
C.texts()

[None, _None_ (None) [Epistolary: Eighteenth-Century_Fiction/richards.02],
 Richardson, Samuel, 1689-1761, _Pamela (1741)_ (1741) [Epistolary: _chadwyck/Eighteenth-Century_Fiction/richards.04],
 Burney, Fanny, 1752-1840, _Evelina (1778)_ (1778) [Epistolary: _chadwyck/Eighteenth-Century_Fiction/burney.01]]

In [15]:
# t = Text(id='_chadwyck/Eighteenth-Century_Fiction/burney.01')
# t.meta

In [17]:
# Chad=load('Chadwyck')
# Chad.init()
# Chad.meta

In [12]:
meta=Chad.load_metadata_file()


In [16]:


# def load_metadata_from_df_or_fn(idf,force=False,**attrs):
#     if type(idf)==str: idf=read_df(idf)
#     if idf is None or not len(idf): return pd.DataFrame()
#     #return df_requiring_id_and_corpus(idf,**attrs)
#     return df_requiring_id(idf,**attrs).fillna('')


# def df_requiring_id(df,idkey='id',fillna='',*x,**y):
#     if df is None or not len(df): return pd.DataFrame(columns=[],index=[]).rename_axis(idkey)
#     if df.index.name==idkey and not idkey in set(df.columns): df=df.reset_index()
#     if not idkey in set(df.columns): df[idkey]=''
#     df[idkey]=df[idkey].fillna('')
#     df[idkey]=[(idx if idx else f'X{i+1:04}') for i,idx in enumerate(df[idkey])]
#     df=df.fillna(fillna) if fillna is not None else df
#     df=df.set_index(idkey)
#     return df

""
id
Eighteenth-Century_Fiction/paltock.01
Eighteenth-Century_Fiction/brookefm.02
Eighteenth-Century_Fiction/mackenzi.01
Eighteenth-Century_Fiction/cleland.01
Eighteenth-Century_Fiction/haywood.07
...
Early_English_Prose_Fiction/ee54010.01
Early_English_Prose_Fiction/ee17010.04
Early_English_Prose_Fiction/ee73020.01


In [5]:
t=C.texts()[1]
t

None, _None_ (None) [Epistolary: _chadwyck/Eighteenth-Century_Fiction/richards.04]

In [10]:
t.source.meta

{'id': 'Eighteenth-Century_Fiction/richards.04'}

In [11]:
# !echo $PATH

'??'

,testing
id,
Eighteenth-Century_Fiction/richards.02,x
_chadwyck/Eighteenth-Century_Fiction/richards.04,y
_chadwyck/Eighteenth-Century_Fiction/burney.01,


'!!'

""
id
Eighteenth-Century_Fiction/richards.02
_chadwyck/Eighteenth-Century_Fiction/richards.04
_chadwyck/Eighteenth-Century_Fiction/burney.01


""
id
Eighteenth-Century_Fiction/richards.02
_chadwyck/Eighteenth-Century_Fiction/richards.04
_chadwyck/Eighteenth-Century_Fiction/burney.01


In [3]:
idx='_chadwyck/Eighteenth-Century_Fiction/richards.04'

In [29]:
C = Epistolary()
C.init()
C.t.meta

{'id': '_chadwyck/Eighteenth-Century_Fiction/burney.01'}

In [37]:
t=C.t
t._meta

{'id': '_chadwyck/Eighteenth-Century_Fiction/burney.01'}

In [41]:
t.source.corpus.init_metadata()

AttributeError: 'Chadwyck' object has no attribute 'init_metadata'

In [21]:
# t.id, t.source.id

('_chadwyck/Eighteenth-Century_Fiction/richards.04',
 'Eighteenth-Century_Fiction/richards.04')

Richardson, Samuel, 1689-1761, _Pamela (1741)_ (1741) [_chadwyck/Eighteenth-Century_Fiction/richards.04]

In [9]:
t.meta

{'testing': 'x', 'id': 'Eighteenth-Century_Fiction/richards.02'}

In [13]:
t.source_text()

epistolary Eighteenth-Century_Fiction/richards.02 --> epistolary Eighteenth-Century_Fiction/richards.02


False

In [23]:
source_text(t)

,author,title,year,idref,pub,type,name,nation,medium,subcorpus,fn_raw,id_corpus,_id,id_text
id,,,,,,,,,,,,,,
Eighteenth-Century_Fiction/richards.01,"Richardson, Samuel, 1689-1761",Clarissa (1st ed.) (1748),1748,Z200044053,Printed for S. Richardson: [etc.],BOOK,Vol. 1,British,Fiction,Eighteenth-Century_Fiction,Eighteenth-Century_Fiction/richards.01.new,chadwyck,chadwyck|Eighteenth-Century_Fiction/richards.01,Eighteenth-Century_Fiction/richards.01


In [8]:
Chad=load('Chadwyck')
Chad.meta

""


In [43]:
# load_metadata_from_df_or_fn(meta, id_corpus_default='chad')
meta_final=pd.DataFrame([
    {'id_text':'Eighteenth-Century_Fiction/richards.01', 'final':'!', 'id_corpus':'chadwyck'},
])
load_metadata_from_df_or_fn(meta_final)
# meta_final

,id_text,final,id_corpus,id
_id,,,,
chadwyck|Eighteenth-Century_Fiction/richards.01,Eighteenth-Century_Fiction/richards.01,!,chadwyck,chadwyck|Eighteenth-Century_Fiction/richards.01


In [44]:
C = Epistolary()


,id,testing,id_text,id_corpus,final
_id,,,,,
Eighteenth-Century_Fiction/richards.02,Eighteenth-Century_Fiction/richards.02,x,Eighteenth-Century_Fiction/richards.02,,
chadwyck|chadwyck|Eighteenth-Century_Fiction/richards.04,chadwyck|Eighteenth-Century_Fiction/richards.04,y,Eighteenth-Century_Fiction/richards.04,chadwyck,
chadwyck|chadwyck|Eighteenth-Century_Fiction/burney.01,chadwyck|Eighteenth-Century_Fiction/burney.01,,Eighteenth-Century_Fiction/burney.01,chadwyck,


In [161]:
# load_metadata_from_df_or_fn(meta)

In [125]:
meta

,id,x
0,chadwyck|Eighteenth-Century_Fiction/richards.01,hello
1,NaN,goodbye


In [106]:
# meta.update??

In [107]:
init_metadata(Epistolary(),meta_init=meta,other_meta=[lltk.load('Chadwyck')])

'2'

""
id
chadwyck|Eighteenth-Century_Fiction/richards.01
chadwyck|Eighteenth-Century_Fiction/richards.04
chadwyck|Eighteenth-Century_Fiction/burney.01


'2'

,x
id,
chadwyck|Eighteenth-Century_Fiction/richards.01,hello
X0002,goodbye


,x
id,
chadwyck|Eighteenth-Century_Fiction/richards.01,hello
chadwyck|Eighteenth-Century_Fiction/richards.04,
chadwyck|Eighteenth-Century_Fiction/burney.01,


,id,x
0,X1,hello
1,NaN,goodbye


'/Users/ryan/lltk_data/corpora/epistolary/metadata.csv'

In [27]:
C=Epistolary()
C.init_metadata()

0       chadwyck|chadwyck|Eighteenth-Century_Fiction/p...
1       chadwyck|chadwyck|Eighteenth-Century_Fiction/b...
2       chadwyck|chadwyck|Eighteenth-Century_Fiction/m...
3       chadwyck|chadwyck|Eighteenth-Century_Fiction/c...
4       chadwyck|chadwyck|Eighteenth-Century_Fiction/h...
                              ...                        
1332    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1333    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1334    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1335    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1336    chadwyck|chadwyck|Early_English_Prose_Fiction/...
Name: id, Length: 1337, dtype: object

0       chadwyck|chadwyck|Eighteenth-Century_Fiction/p...
1       chadwyck|chadwyck|Eighteenth-Century_Fiction/b...
2       chadwyck|chadwyck|Eighteenth-Century_Fiction/m...
3       chadwyck|chadwyck|Eighteenth-Century_Fiction/c...
4       chadwyck|chadwyck|Eighteenth-Century_Fiction/h...
                              ...                        
1332    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1333    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1334    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1335    chadwyck|chadwyck|Early_English_Prose_Fiction/...
1336    chadwyck|chadwyck|Early_English_Prose_Fiction/...
Name: id, Length: 1337, dtype: object

NameError: name 'meta_chad' is not defined

In [26]:
C.path_metadata_init

'/Users/ryan/lltk_data/corpora/epistolary2/metadata_init.csv'

In [25]:
C.load_metadata()

""


In [19]:
C=Epistolary()

In [20]:
C.texts

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [25]:
clary = get_clarissa()
clary = SourceTextEpistolary(clary, hello='x!?')

x!?
Cool! {'id': 'Eighteenth-Century_Fiction/richards.01', 'author': 'Richardson, Samuel, 1689-1761', 'title': 'Clarissa (1st ed.) (1748)', 'year': 1748.0, 'idref': 'Z200044053', 'pub': 'Printed for S. Richardson: [etc.]', 'type': 'BOOK', 'name': 'Vol. 1', 'nation': 'British', 'medium': 'Fiction', 'subcorpus': 'Eighteenth-Century_Fiction', 'fn_raw': 'Eighteenth-Century_Fiction/richards.01.new', 'path_freqs': '/Users/ryan/lltk_data/corpora/chadwyck/freqs/Eighteenth-Century_Fiction/richards.01.json', 'path_txt': '/Users/ryan/lltk_data/corpora/chadwyck/txt/Eighteenth-Century_Fiction/richards.01.txt', 'path_xml': '/Users/ryan/lltk_data/corpora/chadwyck/xml/Eighteenth-Century_Fiction/richards.01.new', '_year_orig': '1748'}


x!?
Cool! {'id': 'Eighteenth-Century_Fiction/richards.01', 'author': 'Richardson, Samuel, 1689-1761', 'title': 'Clarissa (1st ed.) (1748)', 'year': 1748.0, 'idref': 'Z200044053', 'pub': 'Printed for S. Richardson: [etc.]', 'type': 'BOOK', 'name': 'Vol. 1', 'nation': 'British', 'medium': 'Fiction', 'subcorpus': 'Eighteenth-Century_Fiction', 'fn_raw': 'Eighteenth-Century_Fiction/richards.01.new', 'path_freqs': '/Users/ryan/lltk_data/corpora/chadwyck/freqs/Eighteenth-Century_Fiction/richards.01.json', 'path_txt': '/Users/ryan/lltk_data/corpora/chadwyck/txt/Eighteenth-Century_Fiction/richards.01.txt', 'path_xml': '/Users/ryan/lltk_data/corpora/chadwyck/xml/Eighteenth-Century_Fiction/richards.01.new', '_year_orig': '1748'}


[__main__.SourceTextEpistolaryCls,
 Richardson, Samuel, 1689-1761, _Clarissa (1st ed.) (1748)_ (1748.0) [Chadwyck: Eighteenth-Century_Fiction/richards.01]]

TypeError: 'NoneType' object is not callable